#Student  Details

Student Name - Aryank Gupta

Student ID - 24101613

# Task 1

a). Modify your Multinomial Naïve Bayes (MNB) classifier from Etivity5, Task2 to train and test a sentiment classifier.

In [28]:
import math

def multinomialNaiveBayesClassifier(trainingSet, testSet):

    # On Preprocessing Separate positive and negative data
    positive_words = []
    negative_words = []
    for text, label in trainingSet:
        words = text.lower().split()
        if label == '+':
            positive_words.extend(words)
        else:
            negative_words.extend(words)

    # To calculate the word frequencies
    pos_BOW = {word: positive_words.count(word) for word in set(positive_words)}
    neg_BOW = {word: negative_words.count(word) for word in set(negative_words)}

    pos_count = sum(pos_BOW.values())
    neg_count = sum(neg_BOW.values())

    # Vocabulary size
    vocabulary = set(positive_words + negative_words)
    vocab_size = len(vocabulary)

    prob_pos = pos_count / (pos_count + neg_count)
    prob_neg = neg_count / (pos_count + neg_count)

    print(f"posCount={pos_count}")
    print(f"negCount={neg_count}")
    print(f"probPos={prob_pos:.4f}")
    print(f"probNeg={prob_neg:.4f}")
    print(f"pos_BOW={pos_BOW}")
    print(f"neg_BOW={neg_BOW}")
    print(f"Vocabulary={vocabulary}")
    print(f"|V|={vocab_size}")
    print("-" * 40)

    for text, _ in testSet:
        words = text.lower().split()
        doc_prob_pos = math.log(prob_pos)
        doc_prob_neg = math.log(prob_neg)

        print(f"Test document=('{text}', '?')")
        for word in words:
            word_conditional_prob_pos = (pos_BOW.get(word, 0) + 1) / (pos_count + vocab_size)
            word_conditional_prob_neg = (neg_BOW.get(word, 0) + 1) / (neg_count + vocab_size)

            doc_prob_pos += math.log(word_conditional_prob_pos)
            doc_prob_neg += math.log(word_conditional_prob_neg)

            print(f'word= "{word}"  wordConditionalProbPos={word_conditional_prob_pos:.4f}  wordConditionalProbNeg={word_conditional_prob_neg:.4f}')

        inferred_class = '+' if doc_prob_pos > doc_prob_neg else '-'
        print(f"docProbPos={doc_prob_pos:.4f}")
        print(f"docProbNeg={doc_prob_neg:.4f}")
        print(f"Inferred class={inferred_class}")
        print("-" * 40)


trainingSet = [
    ('Boxing scene was a disappointment', '-'),
    ('No plot twists or great scenes', '-'),
    ('Great satire and great plot twists', '+'),
    ('Great scenes a great film', '+')
]

testSet = [('Great disappointment indeed', '?')]

multinomialNaiveBayesClassifier(trainingSet, testSet)


posCount=11
negCount=11
probPos=0.5000
probNeg=0.5000
pos_BOW={'twists': 1, 'film': 1, 'plot': 1, 'and': 1, 'great': 4, 'satire': 1, 'scenes': 1, 'a': 1}
neg_BOW={'twists': 1, 'or': 1, 'no': 1, 'plot': 1, 'boxing': 1, 'great': 1, 'scenes': 1, 'a': 1, 'was': 1, 'scene': 1, 'disappointment': 1}
Vocabulary={'twists', 'film', 'or', 'plot', 'no', 'and', 'great', 'satire', 'scenes', 'boxing', 'a', 'was', 'scene', 'disappointment'}
|V|=14
----------------------------------------
Test document=('Great disappointment indeed', '?')
word= "great"  wordConditionalProbPos=0.2000  wordConditionalProbNeg=0.0800
word= "disappointment"  wordConditionalProbPos=0.0400  wordConditionalProbNeg=0.0800
word= "indeed"  wordConditionalProbPos=0.0400  wordConditionalProbNeg=0.0400
docProbPos=-8.7403
docProbNeg=-8.9635
Inferred class=+
----------------------------------------




b). Copy your solution for (a) into a new code cell.  Modify your code to optimise it for the task of sentiment analysis, i.e., change your classification algorithm from MNB to BMNB (Binary Multinomial Naïve Bayes).

In [29]:
import math

def binaryMultinomialNaiveBayesClassifier(trainingSet, testSet):
    wordsPos = set()
    wordsNeg = set()
    countPos = 0
    countNeg = 0
    pos_BOW = {}
    neg_BOW = {}

    for doc, label in trainingSet:
        unique_words = set(doc.lower().split())  # Convert to lowercase and take unique words
        if label == '+':
            wordsPos.update(unique_words)
            countPos += 1
            for word in unique_words:
                pos_BOW[word] = pos_BOW.get(word, 0) + 1
        elif label == '-':
            wordsNeg.update(unique_words)
            countNeg += 1
            for word in unique_words:
                neg_BOW[word] = neg_BOW.get(word, 0) + 1

    probPos = countPos / (countPos + countNeg)
    probNeg = countNeg / (countPos + countNeg)

    V = wordsPos | wordsNeg
    V_size = len(V)

    print(f'Probability of Positive = {probPos}')
    print(f'Probability of Negative = {probNeg}')
    print(f'Positive BOW = {pos_BOW}, len = {len(pos_BOW)}')
    print(f'Negative BOW = {neg_BOW}, len = {len(neg_BOW)}')
    print(f'Vocabulary (V) = {V}')
    print(f'|V| = {V_size}')
    print('-' * 30)

    for test in testSet:
        doc_text = test[0]
        print(f"Test Document = '{doc_text}'")

        docProbPos = math.log(probPos)
        docProbNeg = math.log(probNeg)

        test_words = set(doc_text.lower().split())

        for word in test_words:
            print(f"Word = '{word}'")
            if word not in V:
                print(f"  Word '{word}' not in vocabulary, skipping.")
                continue

            word_probPos = (pos_BOW.get(word, 0) + 1) / (countPos + V_size)
            word_probNeg = (neg_BOW.get(word, 0) + 1) / (countNeg + V_size)

            docProbPos += math.log(word_probPos)
            docProbNeg += math.log(word_probNeg)

            print(f"  wordConditionalProbPos = {word_probPos:.6f}, wordConditionalProbNeg = {word_probNeg:.6f}")

        finalProbPos = math.exp(docProbPos)
        finalProbNeg = math.exp(docProbNeg)
        print(f'docProbPos = {finalProbPos:.4e} , docProbNeg = {finalProbNeg:.4e}')
        if docProbPos > docProbNeg:
            print(f"Inferred Sentiment: Positive (+)")
        else:
            print(f"Inferred Sentiment: Negative (-)")
        print('-' * 30)

trainingSet = [
    ('Boxing scene was a disappointment', '-'),
    ('No plot twists or great scenes', '-'),
    ('Great satire and great plot twists', '+'),
    ('Great scenes a great film', '+')
]
testSet = [
    ('Great disappointment indeed', '?')
]

binaryMultinomialNaiveBayesClassifier(trainingSet, testSet)

Probability of Positive = 0.5
Probability of Negative = 0.5
Positive BOW = {'twists': 1, 'plot': 1, 'and': 1, 'great': 2, 'satire': 1, 'a': 1, 'scenes': 1, 'film': 1}, len = 8
Negative BOW = {'boxing': 1, 'a': 1, 'was': 1, 'scene': 1, 'disappointment': 1, 'twists': 1, 'or': 1, 'no': 1, 'plot': 1, 'great': 1, 'scenes': 1}, len = 11
Vocabulary (V) = {'film', 'plot', 'satire', 'great', 'a', 'disappointment', 'twists', 'or', 'no', 'and', 'boxing', 'scenes', 'was', 'scene'}
|V| = 14
------------------------------
Test Document = 'Great disappointment indeed'
Word = 'indeed'
  Word 'indeed' not in vocabulary, skipping.
Word = 'great'
  wordConditionalProbPos = 0.187500, wordConditionalProbNeg = 0.125000
Word = 'disappointment'
  wordConditionalProbPos = 0.062500, wordConditionalProbNeg = 0.125000
docProbPos = 5.8594e-03 , docProbNeg = 7.8125e-03
Inferred Sentiment: Negative (-)
------------------------------


# Task 2

Write a Sentiment Analysis function that takes a string as input and identifies its sentiment using the TextBlob library.

In [30]:
from textblob import TextBlob

def sentimentAnalyzer(string):
    # Analyze the sentiment using TextBlob
    analysis = TextBlob(string)
    polarity = analysis.sentiment.polarity
    subjectivity = analysis.sentiment.subjectivity

    # Determine sentiment label
    if polarity > 0:
        sentiment_label = "Positive sentiment 😊"
    elif polarity < 0:
        sentiment_label = "Negative sentiment 😞"
    else:
        sentiment_label = "Neutral sentiment 😐"

    # Print the output in the desired format
    print(f"String= {string}\n")
    print(f"Sentiment(polarity={polarity}, subjectivity={subjectivity})")
    print(sentiment_label)
    print(f"Subjectivity: {subjectivity}")
    print("-" * 33)

# Test strings
test_strings = [
    "NLP is cool",
    "NLP is cool and useful",
    "NLP is hard",
    "NLP is hard and useless",
    "NLP stands for Natural Language Processing"
]

# Run the function for each test string
for sentence in test_strings:
    sentimentAnalyzer(sentence)


String= NLP is cool

Sentiment(polarity=0.35, subjectivity=0.65)
Positive sentiment 😊
Subjectivity: 0.65
---------------------------------
String= NLP is cool and useful

Sentiment(polarity=0.32499999999999996, subjectivity=0.325)
Positive sentiment 😊
Subjectivity: 0.325
---------------------------------
String= NLP is hard

Sentiment(polarity=-0.2916666666666667, subjectivity=0.5416666666666666)
Negative sentiment 😞
Subjectivity: 0.5416666666666666
---------------------------------
String= NLP is hard and useless

Sentiment(polarity=-0.39583333333333337, subjectivity=0.37083333333333335)
Negative sentiment 😞
Subjectivity: 0.37083333333333335
---------------------------------
String= NLP stands for Natural Language Processing

Sentiment(polarity=0.1, subjectivity=0.4)
Positive sentiment 😊
Subjectivity: 0.4
---------------------------------
